<p style="text-align: center;"><span style="color: #ff0000;"><strong><span style="font-size: x-large;">
    ANEXO XXX: ALGORITMOS PKE</span></strong></span></p>

<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Realizado por:</span></strong></span></p>
<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Gabriel Vacaro Goytia</span></strong></span></p>
<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Ignacio Warleta Murcia</span></strong></span></p>

Este notebook contiene una implementación de los algoritmos fundamentales del esquema de cifrado post–cuántico Kyber, en particular el KyberPKE (Public Key Encryption). Kyber es un sistema criptográfico basado en el problema de Redes Lattice y es uno de los candidatos más destacados para ser parte del estándar de criptografía post-cuántica propuesto por el NIST.

Organizamos el anexo según el siguiente índice:

# Índice

1. [Introducción](#1.-Introducción)
2. [Configuración previa](#2.-Configuracion-previa)
3. [Algoritmo de generación de claves](#3.-Algoritmo-de-generacion-de-claves)
4. [Algoritmo de cifrado](#4.-Algoritmo-de-cifrado)
5. [Algoritmo de descifrado](#5.-Algoritmo-de-descifrado)
6. [Ejemplo de uso](#6.-Ejemplo-de-uso)

---
# 1. Introducción






Este notebook cuenta con los 3 algoritmos relativos al PKE de Kyber–KEM:

- Generación de claves (Key Generation) <br>
- Cifrado (Encryption) <br>
- Descifrado (Decryption) <br>

El propósito de esta implementación es comprender los detalles detrás de cada fase del esquema KyberPKE de manera didáctica, con especial énfasis en los aspectos técnicos que permiten asegurar la privacidad y la integridad de la comunicación en un entorno potencialmente afectado por computadoras cuánticas.


---
# 2. Configuracion previa

A continuación, se muestra la configuración previa a ejecutar y los parámetros a definir. En este caso hemos optado por un primo q relativamente pequeño, 743, dado su caracter didáctico, ya que, de esta manera se puede apreciar ligeramente el error que se produce al descifrar el mensaje. Por supuesto, el usuario es libre de cambiar este primo a su gusto y comprobar por su cuenta como se maneja el error.

In [1]:
#MODULOS A IMPORTAR
import numpy as np

In [2]:
# Parámetros básicos
q = 743  # Un número primo pequeño típico en Kyber podría ser 3329
k = 3  # Tamaño del vector/matriz (varía según los estándares Kyber-512, 768, 1024)
mu_1 = 1.0  # Parámetro para la distribución de error más controlado (menor desviación estándar)

---
# 3. Algoritmo de generacion de claves

Primeramente, tenemos algunas funciones auxiliares que nos ayudan a generar una semilla aleatoria que utilizaremos para inicializar y generar la matriz pública A. Esta semilla garantiza que la generación de claves sea determinista y reproducible, pero a la vez, aleatoria.

In [3]:
#Funciones auxiliares 

# Función para generar una semilla aleatoria
def generate_seed():
    return np.random.bytes(16)  # 256 bits de semilla aleatoria

# Función de generación pseudoaleatoria de la matriz A (simplificada)
def generate_matrix_A(seed, k, q):
    np.random.seed(int.from_bytes(seed, "big") % (2**32))
    return np.random.randint(0, q, size=(k, k))

# Función para muestrear errores desde una distribución gaussiana
def sample_error(mu, k, q):
    return np.round(np.random.normal(0, mu, size=(k, 1))).astype(int) % q

A continuacion se explica paso a paso el algoritmo:

1. **Generación de la matriz pública A:**
   En el segundo paso se define A como una matriz pública de tamaño $k∙k$ cuyos elementos son tomados de un conjunto de enteros módulo $q$ (del conjunto $R_q$). PSR es un proceso de muestreo que genera valores aleatorios para A.

2. **Generación de los vectores secreto y de error:**
   En el tercer y cuarto pasos se generan los vectores $s$ y $e$. El vector secreto $s$ es un vector aleatorio de $k$ elementos, cuyos valores provienen de una distribución  $\beta_{\mu_1}$. Esta distribución controla la cantidad de ruido o error en el sistema. De manera similar, el vector $e$ también se genera a partir de la misma distribución. El vector $e$ representa el error que se agrega a la multiplicación de $A$ y $s$ para asegurar la seguridad del sistema.

4. **Cálculo de la clave pública:**
   Seguidamente, se calcula la clave pública $b$ como el resultado de multiplicar la matriz $A$ por el vector secreto $s$ y luego agregar el vector de error $e$. Esto crea un error en el producto para hacer que sea difícil de resolver para un atacante, incluso si conoce la matriz $A$.

5. **Construcción de las claves pública y privada:**
   En los pasos sexto y séptimo se construyen las claves pública y privada. En primer lugar, la clave pública $pk$ se forma concatenando el vector $b$ con la semilla $seed A$. Esto asegura que cualquier persona que conozca $pk$ pueda verificar la autenticidad de $b$ y generar su propia clave compartida, pero no pueda conseguir fácilmente $s$. Por otro lado, la clave privada $sk$ es simplemente el vector secreto $s$.

6. **Devolución de las claves:**
   Por último, el algoritmo devuelve la clave pública $pk$ y la clave privada $sk$.


In [4]:
# Algoritmo 1: Generación de Claves G'
def key_generation():
    seed_A = generate_seed()  # Genera semilla
    A = generate_matrix_A(seed_A, k, q)  # Matriz A en R_q^(k×k)
    
    s = sample_error(mu_1, k, q)  # Vector s con distribución de error
    e = sample_error(mu_1, k, q)  # Vector e con distribución de error

    b = (A @ s + e) % q  # Producto matricial con ruido

    pk = (b, seed_A)  # Clave pública: (b || seed_A)
    sk = s  # Clave secreta: s

    return pk, sk

---
# 4. Algoritmo de cifrado

El algoritmo de cifrado recibe como parámetros de entrada la clave pública $pk$ generada en el paso anterior, y con ella el vector $b$ y la $seed A$ que usará para la generacion de la matriz $A$ tranpuesta. Posteriormente sigue los siguientes pasos:

1. **Generación del vector aleatorio $r$:**  
   Seguidamente, generamos un vector $r$ aleatorio de longitud $k$, cuyos valores se extraen de una distribución $\beta_{\mu_1}$ para generar ruido en el cifrado.

2. **Generación de los vectores de error $e_1$ y $e_2$:**  
   En los pasos cuarto y quinto, generamos los vectores de error $e_1$ y $e_2$.  
   - El vector $e_1$ es de longitud $k$ y se genera de acuerdo con la distribución $\beta_{\mu_2}$, que controla el error en la multiplicación de matrices durante el proceso de cifrado.  
   - El vector $e_2$ es un vector de longitud $1$, generado también a partir de la misma distribución. Este se utilizará en el cálculo del componente de la clave en el mensaje cifrado.

3. **Cálculo de $u$ y $v$:**  
   En los siguientes pasos, calculamos los componentes del mensaje cifrado:  
   - El primer componente, $u$, se calcula multiplicando la matriz $A^T$ por el vector $r$ y luego sumando el vector de error $e_1$. Esto asegura que $u$ dependa de la clave pública y del ruido.  
   - El segundo componente, $v$, se calcula tomando el producto $b$ (la clave pública) con el vector $r$, sumando el error $e_2$ y finalmente añadiendo el mensaje $m$. Esto garantiza que $v$ contenga la información del mensaje cifrado de manera que solo la persona con la clave privada pueda descifrarlo.

4. **Generación del mensaje cifrado final:**  
   Por último, concatenamos $u$ y $v$ como el mensaje cifrado final $c$, el cual devuelve el algoritmo.


In [6]:
# Algoritmo 2: Cifrado E(pk, m, r)
def encrypt(pk, m):
    b, seed_A = pk
    r = sample_error(mu_1, k, q)  # Generar el vector r con errores
    e_1 = sample_error(mu_1, k, q)  # Error en u
    e_2 = sample_error(mu_1, k, q)  # Error en v

    # Reconstruir A^T desde la semilla seed_A
    A = generate_matrix_A(seed_A, k, q)  # Reconstruir A desde la semilla
    A_T = A.T  # Transponer la matriz A para obtener A^T

    # Calcular u = A^T * r + e_1
    u = (A_T @ r + e_1) % q

    # Calcular v = b^T * r + e_2 + m
    v = (b.T @ r + e_2 + m) % q

    # El cifrado es el par (u, v)
    c = (u, v)
    return c, r, e_1, e_2

---
# 5. Algoritmo de descifrado

El propósito del algoritmo de descifrado es recuperar el mensaje original a partir del mensaje cifrado $c$ y la clave secreta $sk$.  
- La clave secreta $sk$ es el vector secreto $s$, que el receptor mantiene de forma privada.  
- El mensaje cifrado $c$, como vimos en el esquema anterior, es el resultado del algoritmo de cifrado y consiste en la concatenación de los componentes $u$ y $v$.

### Pasos del descifrado  

1. **Descomposición del mensaje cifrado:**  
   Primero, se descompone $c$ para obtener $u$ y $v$, los cuales son necesarios para recuperar $m$.  

2. **Recuperación del mensaje original:**  
   Aplicando la ecuación correspondiente y usando la clave secreta $sk$, se obtiene el mensaje original $m$.  


In [7]:
# Algoritmo 3: Descifrado D(sk, c)
def decrypt(sk, c):
    u, v = c
    s = sk  # La clave secreta s

    # Calcular m = v - s^T * u
    m_recovered = (v - (s.T @ u)) % q

    return m_recovered

---
# 6. Ejemplo de uso

A continuación se muestra un ejemplo de uso concatenando los tres algoritmos:

In [11]:
# Ejemplo de uso:

# 1. Generar las claves
public_key, secret_key = key_generation()

# Imprimir claves generadas
print("Clave Pública (pk):")
print("b:", public_key[0])
print("seed_A:", public_key[1])

print("\nClave Secreta (sk):")
print(secret_key)

# 2. Definir y mostrar el mensaje original (m es un vector de tamaño k)
m = np.array([1, 2, 3]).reshape((k, 1))  # Mensaje original (vector columna)
print("\nMensaje original (m):")
print(m)

# 3. Cifrado del mensaje
ciphertext, r, e_1, e_2 = encrypt(public_key, m)

# Imprimir el cifrado y los vectores generados
print("\nCifrado (u, v):")
print("u:", ciphertext[0])
print("v:", ciphertext[1])

print("\nVectores y errores generados durante el cifrado:")
print("r (vector aleatorio r):", r)
print("e_1 (error en u):", e_1)
print("e_2 (error en v):", e_2)

# 4. Descifrar el mensaje
recovered_message = decrypt(secret_key, ciphertext)

# Imprimir el mensaje recuperado
print("\nMensaje recuperado:")
print(recovered_message)


Clave Pública (pk):
b: [[219]
 [522]
 [134]]
seed_A: b'\xdcO\x919EJ\xad \xbfRm\x15\xdah\x9b\xfd'

Clave Secreta (sk):
[[  0]
 [  0]
 [742]]

Mensaje original (m):
[[1]
 [2]
 [3]]

Cifrado (u, v):
u: [[573]
 [365]
 [526]]
v: [[222]
 [223]
 [223]]

Vectores y errores generados durante el cifrado:
r (vector aleatorio r): [[  0]
 [742]
 [  0]]
e_1 (error en u): [[0]
 [0]
 [1]]
e_2 (error en v): [[  0]
 [  0]
 [742]]

Mensaje recuperado:
[[5]
 [6]
 [6]]
